In [18]:
import os
import json
import pyautogui
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys

In [19]:
from src.chrome_manager import select_mobile_and_get_driver
from src.account_manager import login, save_cookies
from src.utils import x_path_click
from resources.xpath_dict import xpath_dict, id_dict
from src.naverband_automation import roof_bands, perform_logout
from config import *

In [20]:
import pyautogui    # prompt 입력에 필요
import time         # sleep, 시간 계산 등에 필요
import random       # rand_sleep, 셔플 등에 필요
from selenium import webdriver   # driver 사용에 필요
from selenium.webdriver.common.alert import Alert   # Alert 처리 필요
from selenium.webdriver.common.keys import Keys     # 키입력(ESC, ENTER)에 필요

from src.chrome_manager import select_mobile_and_get_driver
from src.account_manager import login, save_cookies
from src.utils import x_path_click
from resources.xpath_dict import xpath_dict, id_dict
from src.naverband_automation import roof_bands, perform_logout
from config import *

### mobile, driver = select_mobile_and_get_driver() 부분


In [4]:
# 1) id_dict에서 전화번호 키 리스트 추출
ids = list(id_dict.keys())
print(ids)

['01075381965', '01020664732', '01027851965']


In [5]:
from src.utils import resource_path, ensure_dir, get_cookie_path, x_path_click, x_path_send_keys, logger, move_mouse_naturally
# 2) pyautogui로 전화번호 선택
mobile_num = pyautogui.confirm('전화번호를 선택하시오', buttons=ids)
logger.info(f'선택한 전화번호는 {mobile_num}')

[2025-10-24 02:54:57] [INFO] 선택한 전화번호는 01075381965


In [6]:
from src.utils import get_profile_path
# 3) 선택된 전화번호에 맞는 크롬 프로필 경로 계산 및 생성
profile_dir = get_profile_path(mobile_num)

[2025-10-24 02:55:00] [INFO] get_profile_paht()->profile_dir의 경로는 d:\coding\band_project\accounts/01075381965/chrome_profile


In [7]:
from src.chrome_manager import fix_chrome_profile_preferences
 # 4) 프로필 Preferences 파일 패치
fix_chrome_profile_preferences(profile_dir)

[2025-10-24 02:57:08] [INFO] 크롬 프로필 기본 저장 위치 : d:\coding\band_project\accounts/01075381965/chrome_profile\Default\Preferences


In [9]:
# 5) ChromeOptions 객체 생성 및 옵션 추가
options = webdriver.ChromeOptions()
options.add_argument(f"--user-data-dir={profile_dir}")  # 프로필 적용
logger.info(f'profile적용')

[2025-10-24 03:03:56] [INFO] profile적용


In [10]:
# 로그레벨 설정 (Print 로그 최소화)
options.add_argument('--log-level=3')
logger.info(f'print-log-최소화')

[2025-10-24 03:05:19] [INFO] print-log-최소화


In [11]:
# GPU 비활성화 옵션
options.add_argument('--disable-gpu')
options.add_argument('--disable-software-rasterizer')
logger.info(f'GPU비활성화')

[2025-10-24 03:06:14] [INFO] GPU비활성화


In [12]:
# 탐지 회피 관련 옵션 추가
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
logger.info('탐지관련옵션추가')

[2025-10-24 03:07:00] [INFO] 탐지관련옵션추가


In [15]:
for opt in DEFAULT_CHROME_OPTIONS:
    options.add_argument(opt)
    print(opt) # 기본 옵션
    
    
                               
options.add_argument(f"user-agent={USER_AGENT}")       # 사용자 에이전트

--disable-blink-features=AutomationControlled
--disable-infobars
--disable-extensions
--no-sandbox
--disable-dev-shm-usage
--ignore-certificate-errors
--disable-popup-blocking
--start-maximized


In [16]:

# 5-1)======비밀번호 저장/자동완성 관리 팝업 사전 차단 ======
# 이 프리퍼런스(prefs) 옵션을 추가하면 로그인 후 비밀번호 저장 안내창이 더 이상 나타나지 않음
prefs = {
    "credentials_enable_service": False,            # 크롬 비밀번호 관리 서비스 OFF
    "profile.password_manager_enabled": False       # 프로필 비밀번호 저장/자동완성 OFF
}
options.add_experimental_option("prefs", prefs)

# 6) ChromeDriverManager로 드라이버 설치/업데이트 후 실행
service = Service(ChromeDriverManager().install(), log_path='NUL')  # 윈도우 기준 콘솔 로그 숨김
driver = webdriver.Chrome(service=service, options=options)

In [7]:
# 7) 네이버밴드 로그인 페이지로 자동 이동
driver.get(NAVERBAND_URL)

In [18]:
# 탐지 JS 속성 우회 강화 (execute_cdp_cmd 사용)
try:
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
            Object.defineProperty(navigator, 'webdriver', {get: () => false});
            Object.defineProperty(navigator, 'plugins', {
                get: () => [1, 2, 3, 4, 5],
            });
            Object.defineProperty(navigator, 'languages', {
                get: () => ['ko-KR', 'ko'],
            });
        """
    })
except Exception as e:
        logger.error(f"[WARN] 탐지 JS 우회 코드 주입 실패: {e}")

In [20]:
# 8) 드라이버에 선택된 전화번호 정보 저장
driver.selected_mobile = mobile_num

from src.chrome_manager import close_restore_popup
# 9) "페이지 복원" 팝업 자동 닫기 시도
close_restore_popup(driver)

[2025-10-24 03:18:38] [INFO] [INFO] 복원 팝업 ESC 키 전송으로 닫음


In [21]:
mobile, driver = select_mobile_and_get_driver()

[2025-10-24 03:59:53] [INFO] 선택한 전화번호는 01075381965
[2025-10-24 03:59:53] [INFO] get_profile_paht()->profile_dir의 경로는 d:\coding\band_project\accounts/01075381965/chrome_profile
[2025-10-24 03:59:53] [INFO] fix_chrome_profile_preferences()->크롬 프로필 기본 저장 위치 : d:\coding\band_project\accounts/01075381965/chrome_profile\Default\Preferences
[2025-10-24 03:59:59] [INFO] [INFO] 마우스 자연 이동: (863, 213), duration: 1.42s
[2025-10-24 03:59:59] [INFO] [INFO] 복원 팝업 ESC 키 전송으로 닫음


### login(driver)부분

In [6]:
from selenium.webdriver.support.ui import WebDriverWait
account_id = driver.selected_mobile
wait = WebDriverWait(driver, 10)

In [10]:
from src.account_manager import load_cookies
# 1. 쿠키 인증 복원(도메인별 add_cookie)
load_cookies(driver)
time.sleep(1)

[2025-10-24 03:44:07] [INFO] [쿠키 복원] 현재 도메인/프록시 기능이 활성화되지 않아 쿠키 주입은 건너뜁니다. 환경 구축 후 주석 해제 필요.


In [11]:
# 2. 로그인 페이지로 이동
driver.get(NAVERBAND_LOGIN_URL)
time.sleep(2)

In [ ]:
from selenium.webdriver.common.by import By
from src.utils import x_path_send_keys
from src.utils import move_mouse_naturally
from src.utils import resource_path, ensure_dir, get_cookie_path, x_path_click, x_path_send_keys, logger, move_mouse_naturally
# 3. 전화번호 입력창(XPath) 실존 체크
log_in_xpath = xpath_dict['log_in']
if driver.find_elements(By.XPATH, log_in_xpath):
    x_path_send_keys(driver, log_in_xpath, account_id)
    move_mouse_naturally()
    x_path_send_keys(driver, log_in_xpath, Keys.RETURN)
    logger.info("[INFO] 전화번호 입력 성공")
else:
    logger.warning("[WARN] 입력창 없음(자동 인증/사이트 구조 변경 등)")
    # return

[2025-10-24 03:53:59] [INFO] [INFO] 마우스 자연 이동: (377, 351), duration: 0.86s


NameError: name 'logger' is not defined

In [17]:
from selenium.webdriver.common.by import By
from src.utils import x_path_send_keys
from src.utils import move_mouse_naturally
from src.utils import resource_path, ensure_dir, get_cookie_path, x_path_click, x_path_send_keys, logger, move_mouse_naturally
# 4. 비밀번호 입력도 동일하게 처리
password_xpath = xpath_dict['password']
if driver.find_elements(By.XPATH, password_xpath):
    x_path_send_keys(driver, password_xpath, id_dict[account_id])
    move_mouse_naturally()
    x_path_send_keys(driver, password_xpath, Keys.RETURN)
    logger.info("[INFO] 비밀번호 입력 성공")
else:
    logger.warning("[WARN] 비밀번호 입력창 없음!")
    # return

[2025-10-24 03:56:47] [INFO] [INFO] 마우스 자연 이동: (855, 483), duration: 0.42s
[2025-10-24 03:56:48] [INFO] [INFO] 비밀번호 입력 성공


In [22]:
login(driver)

[2025-10-24 04:00:50] [INFO] [쿠키 복원] 현재 도메인/프록시 기능이 활성화되지 않아 쿠키 주입은 건너뜁니다. 환경 구축 후 주석 해제 필요.
[2025-10-24 04:00:56] [INFO] [INFO] 마우스 자연 이동: (670, 471), duration: 1.02s
[2025-10-24 04:00:57] [INFO] [INFO] 전화번호 입력 성공
[2025-10-24 04:00:59] [INFO] [INFO] 마우스 자연 이동: (614, 340), duration: 0.84s
[2025-10-24 04:01:01] [INFO] [INFO] 비밀번호 입력 성공


In [48]:
def process_band(driver, xpath_dict, i, TXT_DIR, IMAGE_DIR):
    """
    밴드 작업을 사람 행동처럼 자연스럽게 수행
    """
    # 브라우저 창 포커스 유지 (윈도우창 제목 예시)
    focus_window("band")
    i ='//*[@id="content"]/section/div[2]/div/ul/li[2]/div/div/a/div[1]/div/span'
    # 마우스 자연 이동 후 밴드 클릭
# 마우스 자연 이동 후 밴드 클릭
    move_mouse_naturally()
    x_path_click(driver,i)
    human_delay("click")

    # 글쓰기 버튼 클릭 + 고민 시간
    human_delay("thinking")
    safe_xpath_click(driver, xpath_dict['글쓰기_1'])

    # 텍스트 입력 (느린 타이핑 포함)
    write_text_from_folder(driver, xpath_dict['글쓰기_2'], TXT_DIR)
    human_delay("typing")

    # 이미지 업로드 + 대기
    upload_file_from_folder(driver, IMAGE_DIR)
    human_delay("upload")

    # 이미지 첨부 클릭 (마우스 이동 포함)
    move_mouse_naturally()
    x_path_click(driver, xpath_dict['이미지첨부'])
    human_delay("click")

    # 게시하기 클릭 + 생각시간
    x_path_click(driver, xpath_dict['이미지게시'])
    human_delay("thinking")

    # 화면 스크롤 후 홈으로 돌아가기
    driver.execute_script("window.scrollBy(0, window.innerHeight / 3)")
    human_delay("scroll")
    x_path_click(driver, xpath_dict['홈'])
    human_delay("scroll")

In [49]:
process_band(driver, xpath_dict, i, TXT_DIR, IMAGE_DIR)

[2025-10-16 23:58:14] [INFO] [INFO] 창 포커스 및 최대화: ● debug.ipynb - band_project - Visual Studio Code
[2025-10-16 23:58:16] [INFO] [INFO] 마우스 자연 이동: (650, 454), duration: 1.36s
[2025-10-16 23:58:16] [INFO] [DELAY] click 단계: 1.29초 대기
[2025-10-16 23:58:18] [INFO] [DELAY] click 단계: 1.26초 대기
[2025-10-16 23:58:19] [INFO] [DELAY] thinking 단계: 3.92초 대기
[2025-10-16 23:58:23] [INFO] [DELAY] click 단계: 0.76초 대기
[2025-10-16 23:58:34] [WARNING] [WARN] XPATH 클릭 실패: //*[@id="content"]/section/div[2]/div/button
에러: Message: 

[2025-10-16 23:58:34] [INFO] [INFO] 에러 스크린샷 저장: d:\coding\band_project\logs/screenshots\xpath_click_fail_20251016_235834.png
[2025-10-16 23:58:35] [INFO] [INFO] 마우스 자연 이동: (368, 347), duration: 0.45s
[2025-10-16 23:58:35] [INFO] [DELAY] thinking 단계: 3.04초 대기
[2025-10-16 23:58:38] [INFO] [DELAY] click 단계: 1.11초 대기
[2025-10-16 23:58:49] [WARNING] [WARN] 기본 클릭 실패(Message: 
), 인간 패턴 클릭으로 전환 중...
[2025-10-16 23:58:49] [INFO] [DELAY] click 단계: 1.22초 대기
[2025-10-16 23:58:51] [INFO] [DELAY]

[INFO] 본문 자동 입력 성공 (현실적 패턴)


[2025-10-17 00:00:57] [INFO] [DELAY] upload 단계: 3.24초 대기


[INFO] 파일 첨부 성공: d:\coding\band_project\resources/images\mozip1.gif


[2025-10-17 00:01:02] [INFO] [INFO] 마우스 자연 이동: (433, 474), duration: 1.02s
[2025-10-17 00:01:02] [INFO] [DELAY] click 단계: 1.08초 대기
[2025-10-17 00:01:04] [INFO] [DELAY] click 단계: 0.72초 대기
[2025-10-17 00:01:04] [INFO] [DELAY] click 단계: 1.25초 대기
[2025-10-17 00:01:06] [INFO] [DELAY] thinking 단계: 3.49초 대기
[2025-10-17 00:01:10] [INFO] [DELAY] scroll 단계: 0.53초 대기
[2025-10-17 00:01:10] [INFO] [DELAY] click 단계: 1.25초 대기
[2025-10-17 00:01:12] [INFO] [DELAY] scroll 단계: 0.60초 대기
